# HERON Paths

In [1]:
import os
import sys
import operator
import numpy as np

sys.path.append('..')

from ExternalDispatchManager import ExternalDispatchRunner

raven = "~/projects/raven/raven_framework"

RAVEN_LOC = os.path.expanduser(raven)
sys.path.append(os.path.join(RAVEN_LOC, '..'))
sys.path.append(RAVEN_LOC)
sys.path.append('../../raven/ravenframework')

HERON = '~/projects/HERON'
HERON_LOC = os.path.expanduser(HERON)
sys.path.append(os.path.join(HERON_LOC, '..'))
sys.path.append(HERON_LOC)

from src.DispatchManager import DispatchRunner
from ravenframework import ROMExternal
from ravenframework.utils import xmlUtils

c:\Users\sotogj\AppData\Local\Miniconda3\envs\raven_libraries\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sotogj\AppData\Local\Miniconda3\envs\raven_libraries\lib\site-packages\ray\_private\parameter.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
c:\Users\sotogj\AppData\Local\Miniconda3\envs\raven_libraries\lib\site-packages\pkg_resources\__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
2024-07-31 14:47:49,206	IN

## Synthetic Histories

In [2]:
DATA_LOC = os.path.join(HERON_LOC, 'data')
arma_filepath = os.path.join(DATA_LOC,'TSA/Sine/arma.pk')

synth_runner = ROMExternal.ROMLoader(arma_filepath, RAVEN_LOC)
nodes = []
node = xmlUtils.newNode('ROM', attrib={'name': 'SyntheticHistory', 'subType': 'pickledRom'})
node.append(xmlUtils.newNode('clusterEvalMode', text='clustered'))
nodes.append(node)
synth_runner.setAdditionalParams(nodes)

InputData: Using param spec "pickledROM" to read XML node "ROM.


In [3]:
cluster_steps = operator.attrgetter("rom._segmentROM._macroSteps")(synth_runner)
cluster_steps

{10: <ravenframework.SupervisedLearning.ROMCollection.Clusters at 0x1781912e590>,
 12: <ravenframework.SupervisedLearning.ROMCollection.Clusters at 0x1781921e590>,
 11: <ravenframework.SupervisedLearning.ROMCollection.Clusters at 0x1781921f340>}

In [4]:
inp = {'scaling': [1]}
synthHist = synth_runner.evaluate(inp)[0]

(26074630.38 sec) Interp. Cluster ROM      : DEBUG           -> Evaluating interpolated ROM ...
(26074630.38 sec) Interp. Cluster ROM      : DEBUG           ->  ... evaluating macro step "10" (1 / 3)
(26074630.40 sec) Interp. Cluster ROM      : DEBUG           ->  ... evaluating macro step "11" (2 / 3)
(26074630.41 sec) Interp. Cluster ROM      : DEBUG           ->  ... evaluating macro step "12" (3 / 3)


## Dispatch Set Up

In [5]:
DISPATCH_LOC = os.path.join(HERON_LOC, 'src/dispatch')
TEST_LOC = os.path.join(DISPATCH_LOC, 'MRA_testing')
heronlib = os.path.join(TEST_LOC,'testing_stuff/heron.lib')

dispatch_runner = ExternalDispatchRunner()
dispatch_runner.load_heron_lib(heronlib)


c:\Users\sotogj\AppData\Local\Miniconda3\envs\raven_libraries\lib\site-packages\pyutilib\misc\import_file.py:11: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


(            ) UTILS                    : Message         -> importing module C:\Users\sotogj\projects\HERON\tests\integration_tests\mechanics\cashflows\functions


### Mimicking an "outer" to get raven_dict

In [6]:
raven_dict = {}
raven_dict['scaling'] = np.array([1])

In [7]:
# component capacities
for comp in dispatch_runner._components:
    interaction = comp.get_interaction()
    cap = None
    if interaction in comp.get_crossrefs():
        if '_capacity' in comp.get_crossrefs()[interaction]:
            vp = comp.get_crossrefs()[interaction]['_capacity']._vp
            if vp.type == 'FixedValue':
                cap = np.array([vp._parametric])
            else:
                cap = np.array([vp._parametric[0]])
    raven_dict[f'{comp.name}_capacity'] = cap


In [8]:
# which year?
macro_name = dispatch_runner._case.get_year_name()
time_name  = dispatch_runner._case.get_time_name()
# macro_ind = 0
cluster_ind = 0

# signals/Synth Histories
for target, contents in synthHist['_indexMap'][0].items():
    raven_dict[target] = synthHist[target].T

    if set(contents) not in set(raven_dict):
        for t in contents:
            if t == macro_name:
                raven_dict[t] = synthHist[t]
            elif t == time_name:
                raven_dict[t] = synthHist[t]
            else:
                if len(synthHist[t]) > 1:
                    raven_dict[t] = np.array([synthHist[t][cluster_ind]])
                else:
                    raven_dict[t] = np.array([synthHist[t]])

raven_dict['_indexMap'] = synthHist['_indexMap']

In [9]:
raven_vars = dispatch_runner.extract_variables_no_raven(raven_dict)
dispatch, metrics, tot_activity = dispatch_runner.run(raven_vars)

RUNNING HERON DISPATCH MANAGER
Start: 0 End: 21
DEBUGG using solver: glpk
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 0.7654988765716553 s
Start: 0 End: 21
DEBUGG using solver: glpk
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 0.3365046977996826 s
Start: 0 End: 21
DEBUGG using solver: glpk
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 0.3251059055328369 s
Start: 0 End: 21
DEBUGG using solver: glpk
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 0.3154454231262207 s
Start: 0 End: 21
DEBUGG using solver: glpk
DEBUGG sol

C:\Users\sotogj\projects\TEAL\src\main.py:450: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  projCf[decomissionMask] += lifeCf[-1] * taxMult * np.power(inflRate, -1*years[decomissionMask])
